In [13]:
from os.path import dirname, join

import geopandas as gpd
import numpy as np
import pandas as pd
import fiona

from elvis import datasets
from elvis.io.boem_from_file import get_blocks, boem_platform_structures

from elvis.visualization.mapping import (bathymetry_underlay,
                                         create_map_from_geojson,
                                         colors as company_colors,
                                         geojson_underlay)

base_directory = dirname(datasets.__file__)

Make an attribute for "proximity" to infrastructure; by block as a time series (based on install date). 
To simplify things we'll bin my quarter (three month period).

### Remove data less than 30-years old. 

In [14]:
period_size = 'Q'

In [15]:
blocks = get_blocks(base_directory)
blocks.set_index("AREABLK", inplace=True)

In [16]:

platform_structures = boem_platform_structures(base_directory)
platform_structures["AREABLK"] = platform_structures["AREABLK"].str.replace("\W","")

#
platform_structures["Install Period"] = platform_structures["Install Date"].dt.to_period(period_size)
platform_structures["Removal Period"] = platform_structures["Removal Date"].dt.to_period(period_size)

date_range = pd.date_range(start=platform_structures["Install Date"].min(), 
                           end=pd.datetime.now(), freq=period_size)
periods = date_range.to_period(period_size)

In [17]:
# go forware
has_infra = set()
has_infra_by_quarter = {}

for period in periods:
    #
    iindx = platform_structures["Install Period"] == period
    if np.any(iindx):
        has_infra = has_infra.union(set(platform_structures[iindx]["AREABLK"].values))
    
    rindx = platform_structures["Removal Period"] == period
    if np.any(rindx):
        has_infra = has_infra - set(platform_structures[rindx]["AREABLK"].values)
    
    has_infra_by_quarter[period] = has_infra

Places with infrastructure

In [18]:
blk = blocks.loc[has_infra_by_quarter[pd.Period('1974Q2', 'Q-DEC')],:]

create_map_from_geojson(blk.__geo_interface__,
                        color=company_colors["equinor"],
                        control=True)

Map(center=[26.9792212296875, -91.87030927187499], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [19]:
blk = blocks.loc[has_infra_by_quarter[pd.Period('2000Q4', 'Q-DEC')],:]

create_map_from_geojson(blk.__geo_interface__,
                        color=company_colors["equinor"],
                        control=True)

/Users/blasscock/.edm/envs/Equinor/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Map(center=[26.9792212296875, -91.87030927187499], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [20]:
blk = blocks.loc[has_infra_by_quarter[pd.Period('2019Q4', 'Q-DEC')],:]

create_map_from_geojson(blk.__geo_interface__,
                        color=company_colors["equinor"],
                        control=True)

Map(center=[26.9792212296875, -91.87030927187499], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [21]:
from sklearn.neighbors import KernelDensity

def infrastructure_score(blocks, period=pd.Period('1974Q2', 'Q-DEC')):
    blk = blocks.loc[has_infra_by_quarter[period],:]
    blk = blk[~blk.index.isna()].copy()
    
    # centers of every block
    centers = [np.deg2rad(i.centroid.coords.xy) for i in blocks['geometry'].values]
    long = [i[0][0] for i in centers]
    lati = [i[1][0] for i in centers]
    X = np.array((lati, long)).T

    # centers for every block that has well infrastructure at specific time:
    centers = [np.deg2rad(i.centroid.coords.xy) for i in blk['geometry'].values]
    long = [i[0][0] for i in centers]
    lati = [i[1][0] for i in centers]
    Xp = np.array((lati, long)).T

    # haversine give "great circle distance"
    kde = KernelDensity(bandwidth=0.001, metric="haversine")
    kde.fit(Xp)
    
    blocks["id"] = blocks.index
    _infra_score = kde.score_samples(X)

    indx = _infra_score > -10
    blks = blocks[indx]
    infra_score = {key:val for key,val in zip(blks["id"].values, _infra_score[indx])}
    
    # return updated blocks dataframe with the score:
    return blks, infra_score



In [22]:
from ipyleaflet import (basemap_to_tiles, basemaps, Choropleth, GeoJSON, Map)
from branca.colormap import linear

period = pd.Period('1974Q2', 'Q-DEC')
blks, infra_score = infrastructure_score(blocks, period=period)

layer = Choropleth(
    geo_data=blks.__geo_interface__,
    choro_data=infra_score,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.5, 'weight' : 0})

center = (26.9792212296875, -91.87030927187499)
m = Map(center=center, 
                   layers=(basemap_to_tiles(basemaps.Esri.WorldStreetMap,),),
                   zoom = 6)

m.add_layer(layer)

blk = blocks.loc[has_infra_by_quarter[period],:]
x = GeoJSON(data=blk.__geo_interface__,
            style={'fillOpacity':0.25, 'color':'black'})
m.add_layer(x)

m

Map(center=[26.9792212296875, -91.87030927187499], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [23]:
from ipyleaflet import (basemap_to_tiles, basemaps, Choropleth, GeoJSON, Map)
from branca.colormap import linear

period = pd.Period('2000Q4', 'Q-DEC')
blks, infra_score = infrastructure_score(blocks, period=period)

layer = Choropleth(
    geo_data=blks.__geo_interface__,
    choro_data=infra_score,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.5, 'weight' : 0})

center = (26.9792212296875, -91.87030927187499)
m = Map(center=center, 
                   layers=(basemap_to_tiles(basemaps.Esri.WorldStreetMap,),),
                   zoom = 6)

m.add_layer(layer)

blk = blocks.loc[has_infra_by_quarter[period],:]
x = GeoJSON(data=blk.__geo_interface__,
            style={'fillOpacity':0.25, 'color':'black'})
m.add_layer(x)

m

/Users/blasscock/.edm/envs/Equinor/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/Users/blasscock/.edm/envs/Equinor/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Map(center=[26.9792212296875, -91.87030927187499], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [12]:
from ipyleaflet import (basemap_to_tiles, basemaps, Choropleth, GeoJSON, Map)
from branca.colormap import linear

period = pd.Period('2019Q4', 'Q-DEC')
blks, infra_score = infrastructure_score(blocks, period=period)

layer = Choropleth(
    geo_data=blks.__geo_interface__,
    choro_data=infra_score,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.5, 'weight' : 0})

center = (26.9792212296875, -91.87030927187499)
m = Map(center=center, 
                   layers=(basemap_to_tiles(basemaps.Esri.WorldStreetMap,),),
                   zoom = 6)

m.add_layer(layer)

blk = blocks.loc[has_infra_by_quarter[period],:]
x = GeoJSON(data=blk.__geo_interface__,
            style={'fillOpacity':0.25, 'color':'black'})
m.add_layer(x)

m

Map(center=[26.9792212296875, -91.87030927187499], controls=(ZoomControl(options=['position', 'zoom_in_text', …

Infrastructure lead exploration: Implement (min distance to thing).
20 miles "tie" back; can add subsea infrastructure. Bathymetry.

For leases "concentration/density" of leases more important for exploration.